In [ ]:
# d/st model and predictions

In [1]:
import pandas as pd

In [2]:
stats = pd.read_csv("ffb_stats.csv")

In [3]:
stats.columns

Index(['Unnamed: 0', 'Player', 'Tm', 'Pos', 'Passing Cmp', 'Passing Att',
       'Passing Yds', 'Passing TD', 'Int', 'Year', 'Rushing Att',
       'Rushing Yds', 'Rushing TD', 'Rushing Fmb', 'Tgt', 'Rec',
       'Receiving Yds', 'Receiving TD', 'Receiving Fmb', 'Touch',
       'Scrimmage Yards', 'PA', 'Yds Allow', 'TO', 'FL', 'Def Int', 'Pen',
       'EXP', 'PR TD', 'KR TD', 'FblTD', 'IntTD', 'Sfty', 'Short FGA',
       'Short FGM', 'Reg FGA', 'Reg FGM', 'Med FGA', 'Med FGM', 'Long FGA',
       'Long FGM', 'Deep FGA', 'Deep FGM', 'XPA', 'XPM', 'Pts', 'Rk'],
      dtype='object')

In [4]:
feats = ['Player','Tm','Pos','Year','Pts','PA','Yds Allow','Def Int','PR TD', 'KR TD', 'FblTD', 'IntTD', 'Sfty']

In [5]:
ffb_dst = stats[stats['Pos'] == 'D/ST']

In [6]:
dst = ffb_dst[feats]

In [7]:
predictors = ['PA','Yds Allow','Def Int','PR TD', 'KR TD', 'FblTD', 'IntTD', 'Sfty']

In [8]:
train = dst[dst['Year'] < 2022]
test = dst[dst['Year'] == 2022]

In [10]:
from sklearn.linear_model import Ridge

dstm = Ridge(alpha=.1)

In [11]:
dstm.fit(train[predictors], train["Pts"])

Ridge(alpha=0.1)

In [12]:
predictions = dstm.predict(test[predictors])

In [13]:
predictions = pd.DataFrame(predictions, columns=["Pred Pts"], index=test.index)

In [14]:
comb = pd.concat([test[["Player", "Pts"]], predictions], axis=1)

In [15]:
comb

,Player,Pts,Pred Pts
1881,Houston Texans,84.0,80.102773
2090,Dallas Cowboys,75.0,61.608426
2172,New England Patriots,72.0,68.172551
2239,San Francisco 49ers,70.0,70.751087
2286,Philadelphia Eagles,68.0,66.754153
2499,Buffalo Bills,62.0,62.378714
2543,Seattle Seahawks,61.0,57.076427
2570,Cincinnati Bengals,60.0,57.627740
2574,Indianapolis Colts,60.0,54.420673
2631,Denver Broncos,58.0,60.962172


In [16]:
from sklearn.metrics import mean_squared_error

In [17]:
mean_squared_error(comb["Pts"], comb["Pred Pts"])

28.896764199141263

In [18]:
import pickle

In [19]:
with open('dstm.pkl', 'wb') as model_file:
    pickle.dump(dstm, model_file)

In [20]:
import numpy as np
from scipy.stats import norm, beta

In [41]:
player_data = ffb_dst.groupby('Player').agg({
    'PA': ['mean', 'std'],
    'Yds Allow': ['mean', 'std'],
    'Def Int': ['mean', 'std'],
    'PR TD': ['mean', 'std'],
    'KR TD': ['mean', 'std'],
    'FblTD': ['mean', 'std'],
    'IntTD': ['mean', 'std'],
    'Sfty': ['mean', 'std'],
})

In [42]:
num_simulations = 1000

In [43]:
simulated_points = []

for player, stats in player_data.iterrows():
    for _ in range(num_simulations):
        sim_data = {
            'PA': np.random.normal(stats[('PA', 'mean')], stats[('PA', 'std')]),
            'Yds Allow': np.random.normal(stats[('Yds Allow', 'mean')], stats[('Yds Allow', 'std')]),
            'Def Int': np.random.normal(stats[('Def Int', 'mean')], stats[('Def Int', 'std')]),
            'PR TD': np.random.normal(stats[('PR TD', 'mean')], stats[('PR TD', 'std')]),
            'KR TD': np.random.normal(stats[('KR TD', 'mean')], stats[('KR TD', 'std')]),
            'FblTD': np.random.normal(stats[('FblTD', 'mean')], stats[('FblTD', 'std')]),
            'IntTD': np.random.normal(stats[('IntTD', 'mean')], stats[('IntTD', 'std')]),
            'Sfty': np.random.normal(stats[('Sfty', 'mean')], stats[('Sfty', 'std')]),
        }
        
        points = dstm.predict([list(sim_data.values())])[0]
        simulated_points.append({'Player': player, 'Pred Pts': points})

/Users/elijahmclaughlin/anaconda3/lib/python3.10/site-packages/sklearn/base.py:420: UserWarning: X does not have valid feature names, but Ridge was fitted with feature names
  warnings.warn(
/Users/elijahmclaughlin/anaconda3/lib/python3.10/site-packages/sklearn/base.py:420: UserWarning: X does not have valid feature names, but Ridge was fitted with feature names
  warnings.warn(
/Users/elijahmclaughlin/anaconda3/lib/python3.10/site-packages/sklearn/base.py:420: UserWarning: X does not have valid feature names, but Ridge was fitted with feature names
  warnings.warn(
/Users/elijahmclaughlin/anaconda3/lib/python3.10/site-packages/sklearn/base.py:420: UserWarning: X does not have valid feature names, but Ridge was fitted with feature names
  warnings.warn(
/Users/elijahmclaughlin/anaconda3/lib/python3.10/site-packages/sklearn/base.py:420: UserWarning: X does not have valid feature names, but Ridge was fitted with feature names
  warnings.warn(
/Users/elijahmclaughlin/anaconda3/lib/python3

In [44]:
simulated_df = pd.DataFrame(simulated_points)

In [45]:
avg_simulated_points = simulated_df.groupby('Player')['Pred Pts'].mean().reset_index()

In [46]:
dst_proj = avg_simulated_points.sort_values(by='Pred Pts', ascending=False)

In [47]:
dst_proj

,Player,Pred Pts
12,Houston Texans,72.999052
21,New England Patriots,65.833472
22,New Orleans Saints,65.243188
13,Indianapolis Colts,64.397836
18,Los Angeles Rams,62.779308
6,Cincinnati Bengals,61.431979
28,Seattle Seahawks,60.627945
30,Tennessee Titans,60.064269
3,Buffalo Bills,59.614644
25,Philadelphia Eagles,58.697488


In [48]:
dst_proj = dst_proj.sort_values("Pred Pts", ascending=False)
dst_proj["Pred Rk"] = list(range(1,dst_proj.shape[0]+1))

In [49]:
dst_proj

,Player,Pred Pts,Pred Rk
12,Houston Texans,72.999052,1
21,New England Patriots,65.833472,2
22,New Orleans Saints,65.243188,3
13,Indianapolis Colts,64.397836,4
18,Los Angeles Rams,62.779308,5
6,Cincinnati Bengals,61.431979,6
28,Seattle Seahawks,60.627945,7
30,Tennessee Titans,60.064269,8
3,Buffalo Bills,59.614644,9
25,Philadelphia Eagles,58.697488,10


In [40]:
dst_proj.to_csv('proj_dst.csv')